In [2]:
%autosave 120

Autosaving every 120 seconds


In [27]:
import pandas as pd
import spacy
#pip install nltk
from nltk.tokenize import word_tokenize
import numpy as np
import en_core_web_sm


# Load data

In [9]:
train.head(10)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
5,59859,0.666667,ur a sh*tty comment.,0.047619,0.638095,0.000000,0.333333,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.009524,0,105
6,59861,0.457627,hahahahahahahahhha suck it.,0.050847,0.305085,0.000000,0.254237,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.220339,0,59
7,59863,0.000000,FFFFUUUUUUUUUUUUUUU,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
8,239575,0.000000,The ranchers seem motivated by mostly by greed...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,26662,approved,0,0,0,0,0,0.000000,0,4
9,239576,0.000000,It was a great show. Not a combo I'd of expect...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,26650,approved,0,0,0,1,0,0.000000,0,4


In [8]:
train = pd.read_csv(r'data/train.csv')
X_train = train.iloc[:5000,].comment_text

In [7]:
X_train.shape

(5000,)

# Pre-process the text

In [21]:
import re


def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

#print(deEmojify(text))

In [24]:
def preprocess(corpus):
    '''This pre-processing function lemmatizes words, makes a commment case-insensitive, removes punctuation,
    removes stop words and removes single characters. '''
    
    # convert df to list; *corpus* is of a form of a list of str
    corpus = corpus.to_list()
    
    # initialize lemmatizer and a list of stopwords
    lemmatizer = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])
    all_stopwords = lemmatizer.Defaults.stop_words
    
    for i in range(len(corpus)):
        text = corpus[i] # one comment
        
        # convert text to lower case
        text = text.lower()
        
        # remove emoji
        text = deEmojify(text)
        
        # remove url links
        #text = 
        
        # remove punctuations and numbers but save the "'"
        symbols = "—…,!\"#$%&()*+-./:;<=>?@[\]^_`{|}~0123456789"
        for sym in symbols:
            text = np.char.replace(text, sym, ' ') # replace symbols by one space

        # tokenize text
        text_tokens = word_tokenize(str(text))
        
        # remove stop words ("sw") and single characters
        tokens_without_sw = " ".join([word for word in text_tokens if (word not in all_stopwords) and (len(word) > 1)])

        # remove "’"
        tokens_without_sw = np.char.replace(tokens_without_sw, "'", ' ')
        
        # spaCy Lemmatization
        doc = lemmatizer(str(tokens_without_sw))
        corpus[i] = " ".join([token.lemma_ for token in doc]) # Extract the lemma for each token and join
    return corpus

In [25]:
X_train_tidy = preprocess(X_train)
X_train_tidy

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.